# irodsConnector 

## Create an iRODS session (connection to iRODS server)

In [1]:
from ibridges import Session
import os, json
from getpass import getpass

### Password authentication

In [2]:
with open(os.path.expanduser("~/.irods/irods_environment.json"), "r") as f:
    ienv = json.load(f)
password = getpass("Your iRODS password")
session = Session(irods_env=ienv, password=password)

Your iRODS password········
Auth with password


### Using the cached password ~/.irods/.rodsA

In [ ]:
session = Session(irods_env_path=os.path.expanduser("~/.irods/irods_environment.json"))

### Checking some session parameters

In [3]:
print(session.username)
print(session.default_resc) # the resource to which data will be uploaded
print(session.zone) # default home for iRODS /zone/home/username
print(session.server_version)

c.staiger@uu.nl
irodsResc
nluu12p
(4, 2, 12)


## Working with data

In [4]:
from ibridges.irodsconnector.data_operations import DataOperations
from ibridges.utils.path import IrodsPath
data_ops = DataOperations(session)

### Create a collection

On a local file name we have folders/directories and files. In iRODS we call them collections and data objects. In contrast to files, data objects carry system metadata and user defined metadata. Likewise for collections.

In [5]:
# build the path to yur home collection on the iRODS server
irods_home_path = ienv.get('irods_home', '/'+session.zone+'/home')
print(irods_home_path)

/nluu12p/home/research-test-christine


In [6]:
# extend the home path with a new sub collection
irods_path = IrodsPath(session, irods_home_path, 'new_coll')

# assure that the collection does not already exist
print(irods_path.collection_exists())

# create the collection or simply retrieve the collection if it already existed 
coll = data_ops.create_collection(irods_path)

True


Some information about the collection like name, path and data objects in the collection, cumulative size:

In [18]:
print('collection name\t', coll.name)
print('collection path\t', coll.path)
print('data objects\t', coll.data_objects)
print('cumulative size\t', data_ops.get_size(coll))

collection name	 new_coll
collection path	 /nluu12p/home/research-test-christine/new_coll
data objects	 []
cumulative size	 0


### Upload a file or folder

In [8]:
# create a local file
local_path = os.path.expanduser("~/demofile.txt")
f = open(local_path, "a")
f.write("My content! Super important")
f.close()

# upload the file to our collection
data_ops.upload(local_path, irods_path, overwrite = True)

# list again the data object in te collection
print('data objects\t', coll.data_objects)
print('cumulative size\t', data_ops.get_size(coll))

data objects	 [<iRODSDataObject 24295784 demofile.txt>]
cumulative size	 297


### Retrieve a  data object from iRODS and checking their status

In [14]:
obj_path = IrodsPath(session, coll.path, 'demofile.txt')
obj = data_ops.get_dataobject(obj_path)

Some system metadata of the object:

In [15]:
print('data object name\t', obj.name)
print('data object path\t', obj.path)
print('data object size\t', obj.size)
print('data object checksum\t', obj.checksum)
print('data replicas\t', data_ops.obj_replicas(obj))

data object name	 demofile.txt
data object path	 /nluu12p/home/research-test-christine/new_coll/demofile.txt
data object size	 11
data object checksum	 None
data replicas	 [(0, 'lp0054_02', None, 11, 'good')]


### Open a data object in read or write mode

In [11]:
stream = obj.open('r')
text = stream.read().decode()
stream.close()
print(text)

My content! Super importantMy content! Super importantMy content! Super importantMy content! Super importantMy content! Super importantMy content! Super importantMy content! Super importantMy content! Super importantMy content! Super importantMy content! Super importantMy content! Super important


In [12]:
# overwrite the content of the data object
stream = obj.open('w')
stream.write('My new text'.encode())
stream.close()

In [13]:
# check the changes
with obj.open('r') as objRead:
    print(objRead.read().decode())

My new text


### Download a data object or collection

In [16]:
from pathlib import Path
local_path = Path(os.path.expanduser("~/Downloads"))
print("Download folder exists", local_path.is_dir())
data_ops.download(coll.path, local_path)

Download folder exists True


/Users/staig001/git-repos/iBridges/ibridges/irodsconnector/data_operations.py:262: UserWarning: Download: File already exists
	Skipping /nluu12p/home/research-test-christine/new_coll/demofile.txt
  warnings.warn(f'Download: File already exists\n\tSkipping {o}')


### Delete a data object or collection

In [17]:
obj.unlink()
coll.remove()
data_ops.remove(<path>)

## User defined metadata of data objects and collections

In [19]:
# upload a file
local_path = os.path.expanduser("~/demofile.txt")
irods_path = IrodsPath(session, irods_home_path, 'new_coll')
data_ops.upload(local_path, irods_path, overwrite = True)

### Retrieve an iRODS object or collection and list its metadata

In [21]:
obj = data_ops.get_dataobject(str(irods_path) + '/demofile.txt') # TODO: exchange once data_ops is done

In [22]:
from ibridges.irodsconnector.meta import MetaData
obj_meta = MetaData(obj)
print(obj_meta)

 - {name: org_replication_scheduled, value: irodsResc,irodsRescRepl, units: None}
 - {name: org_revision_scheduled, value: irodsResc, units: None}



### View, add, set and delete metadata

In [26]:
obj_meta.add('NewKey', 'NewValue')
obj_meta.add('NewKey', 'AnotherValue')
print(obj_meta)

ValueError: ADD META: Metadata already present

Note, that keys are always capitalised. This is good practice in iRODS.

In [27]:
obj_meta.delete('NEWKEY', 'NewValue')
print(obj_meta)

 - {name: NEWKEY, value: AnotherValue, units: None}
 - {name: org_replication_scheduled, value: irodsResc,irodsRescRepl, units: None}
 - {name: org_revision_scheduled, value: irodsResc, units: None}



We can also set the meta data to a single key, value, units pair. This will remove any other entries with the same key.

In [29]:
obj_meta.set("NEWKEY", "YetAnotherValue")
print(obj_meta)

 - {name: NEWKEY, value: YetAnotherValue, units: None}
 - {name: org_replication_scheduled, value: irodsResc,irodsRescRepl, units: None}
 - {name: org_revision_scheduled, value: irodsResc, units: None}



### Accessing metadata 
With the orint function you can quickly inspect the metadata of an iRODS collection or object. If you want to extract and do something with the metadata, use the `__iter__` function. We give a small example below where we assume that the metadata contains a key/name *AUTHOR*:

In [36]:
for md in obj_meta.__iter__():
    print(md.name, "has value", md.value)

org_revision_scheduled has value irodsResc
NEWKEY has value YetAnotherValue
org_replication_scheduled has value irodsResc,irodsRescRepl


## Resources and handling resources

In [37]:
from ibridges.irodsconnector.resources import Resources
resources = Resources(session)

### Check if default resource exists

In [38]:
default_resc = resources.get_resource(session._irods_env.get("irods_default_resource", ""))
print(default_resc.name)
print(default_resc.free_space) # Metadata how much bytes are left on resource, None if not set
print(default_resc.type) # Storage policy

irodsResc
None
random


### Listing resources

In [39]:
print(resources.resources()) # all resources
print(resources.root_resources) # all writeable resources (name, status, free space, context)

{'bundleResc': {'parent': None, 'status': None, 'context': None, 'free_space': 0}, 'irodsResc': {'parent': None, 'status': None, 'context': None, 'free_space': 0}, 'irodsRescRepl': {'parent': None, 'status': 'up', 'context': None, 'free_space': 0}, 'lp0054': {'parent': '10036', 'status': None, 'context': None, 'free_space': 0}, 'lp0054_02': {'parent': '17412674', 'status': None, 'context': None, 'free_space': 0}, 'lp0054_03': {'parent': '21368707', 'status': None, 'context': None, 'free_space': 0}, 'lp0054_1': {'parent': None, 'status': None, 'context': None, 'free_space': 0}, 'lp0054_p02': {'parent': '10034', 'status': None, 'context': None, 'free_space': 0}, 'lp0054_p03': {'parent': '10034', 'status': None, 'context': None, 'free_space': 0}, 'lp0054_p1': {'parent': None, 'status': None, 'context': None, 'free_space': 0}, 'lp0055': {'parent': '10037', 'status': None, 'context': None, 'free_space': 0}, 'lp0055_02': {'parent': '17421267', 'status': None, 'context': None, 'free_space': 0

### Retrieve current free space
In contrast to `resc.free_space` the function `get_free_space` accumulates all free space in the subtree starting with the resource as parent.

In [40]:
resources.get_free_space(session._irods_env.get("irods_default_resource", "")) # default resource name

0

## Tickets (access string to collection or data object)

### List all tickets which you issued

In [41]:
from ibridges.irodsconnector.tickets import Tickets
tickets = Tickets(session)
print(tickets.all_ticket_strings)
print(tickets.all_tickets()) # (ticket string, access mode, object or collection id, expiry date in epoche)

[]
[]


### Issue a ticket

In [42]:
from datetime import datetime

exp_date = datetime.today().strftime('%Y-%m-%d.%H:%M:%S')
objPath = "/nluu12p/home/research-test-christine/books/BenHur.txt"
ticket = tickets.create_ticket(obj_path=objPath, ticket_type="write", expiry_string=exp_date) # allow write access

In [43]:
tickets.all_ticket_strings

['l5FFm9KJxhD45ma']

### Fetch and delete a ticket

In [44]:
ticket = tickets.get_ticket(tickets.all_ticket_strings[0])
tickets.delete_ticket(ticket)

## Rules
Execute an iRODS rule from a rule file:

In [45]:
from ibridges.irodsconnector.rules import Rules
rules = Rules(session)
rule_file = "example_rules/example.r"
stdout, stderr = rules.execute_rule(rule_file, {})

In [46]:
print(stdout)
print(stderr)

Input for this rule: This is a string or a path or etc
Test output for errors.
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

### Overwrite parameters in iRODS rules
iRODS rule files end with a line like `input *in="This is a string or a path or etc"`. In this example there is an input parameter called `'*in'` and it takes the value `"This is a string or a path or etc"`. We can overwrite these values by passing a python dictionary:

In [47]:
params = {'*in': '"Another input"'}
stdout, stderr = rules.execute_rule(rule_file, params)
print(stdout)

Input for this rule: Another input


Changing the type of the parameter from str to int, you can also see that keys in the dictionary which do not correspond to an input parameter, are simply ignored.

In [48]:
params = {'*in': 4, '*another_val': '"Value"'}
stdout, stderr = rules.execute_rule(rule_file, params)
print(stdout)

Input for this rule: 4


## Permissions

### Accessing the permissions of a data object or collection in iRODS

Objects and collections have permissions attached to them. Permissions, which work like access levels, must be specified per user or group. The basic permissions are `own` (implies reading and writing), `modify object` (editing and reading), and `read object`.

In [49]:
from ibridges.irodsconnector.permissions import Permissions

# select a file to inspect and set permissions on
item_path = ienv.get('irods_home', '/'+session.zone+'/home') # Path to collection or data object
item = data_ops.get_collection(item_path) # TODO: exchange once data_ops is done

# instantiate permissions with that object
perm = Permissions(session, item)
print(f'Permissions for {item_path}:\n')
print(perm)

SyntaxError: expected ':' (permissions.py, line 17)

### Available permissions on your iRODS server

In [ ]:
obj_perm.available_permissions.codes

### Adding permissions to a collection or data object

In [ ]:
perm.set('modify object', '<username or group name>')
print(perm)

Note that some permission-types have synonyms:

+ read object: 'read', 'read object', 'read_object'
+ modify object: 'write', 'modify object', 'modify_object'

### Removing permissions

In [ ]:
perm.set('null', '<username or group name>')
print(perm)

### Inheritance

Collections have two special permissions level `inherit` and `noinherit`. From the point in time where inheritance in switched on, all newly added subcollections and data objects will inherit their initial permissions from the collection.

In [ ]:
# Retrieve a collection from iRODS
coll_path = ienv.get('irods_home1', '/'+session.zone+'/home')
coll = session.irods_session.collections.get(item_path)
coll_perm = Permissions(session, coll)

#Switch inheritance on
coll_perm.set('inherit')
print(coll_perm)

In [ ]:
# Switch inheritance off
coll_perm.set('noinherit')
print(coll_perm)